Minimal shocks retirement model
===========================

This notebook presents the step by step solution for a backward induction problem with a discrete and continuout choice variable for one period based on model_retirement.m

The comparison file for verification of the correct output of each step is retirement_minimal_shocks.m

In [1]:
import numpy as np
import math
import scipy.stats as scps
import matplotlib.pyplot as plt
import pickle

In [2]:
import numpy as np
from collections import namedtuple
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.stats import norm
from copy import *
from numpy.matlib import * 
from scipy.optimize import *

In [3]:
# There potentially is a pythonic solution that would make this function obsolete
def quadrature(n, lbnd, ubnd):
    
    x1 = lbnd
    x2 = ubnd
    x = np.zeros(n)
    w = x
    EPS = 3e-14
    m = int(round((n+EPS)/2)) # flor function in matlab, rounding to the lower integer
    xm = (x2+x1)/2
    xl = (x2-x1)/2
    z1 = 1e99

    x = np.full(n+1, np.nan)
    w = np.full(n+1, np.nan)

    i = 1

    while i <= m:

        z = math.cos(math.pi*(i - 0.25)/(n + 0.5))

        while abs(z - z1) > EPS:
            p1 = 1
            p2 = 0
            j = 1

            while j <= n:
                p3 = p2
                p2 = p1
                p1 = ((2*j -1)*z*p2 - (j-1)*p3)/j
                j += 1

            pp = n*(z*p1 - p2)/(z*z - 1)
            z1 = z
            z = z1 - p1/pp

        x[i] = xm - xl*z
        x[n + 1 - i] = xm + xl*z
        w[i] = 2*xl/((1-z*z)*pp*pp)
        w[n + 1 - i] = w[i]
        i += 1

    x = x[1:]
    w = w[1:]

    return x, w

In [4]:
# Model parameters (default)

Tbar = 25 # number of periods (fist period is t=1) 
ngridm = 500 # number of grid points over assets
mmax = 50 # maximum level of assets
expn = 5 # number of quadrature points used in calculation of expectations
nsims = 10 # number of simulations
init = [10, 30] # interval of the initial wealth
r = 0.05 # interest rate
df = 0.95 # discount factor
sigma = 0.25 # sigma parameter in income shocks
duw = 0.35 #disutility of work
theta = 1.95 # CRRA coefficient (log utility if ==1)
inc0 = 0.75 # income equation: constant
inc1 = 0.04 # income equation: age coef
inc2 = 0.0002 # income equation: age^2 coef
cfloor =0.001 # consumption floor (safety net in retirement)
lambda_ = 0.02 # scale of the EV taste shocks 

In [5]:
# Functions: utility and budget constraint

def util(consumption, working):
    """CRRA utility"""
    
    u = (consumption**(1-theta)-1)/(1-theta)
    u = u - duw*(working)
    
    return u

def mutil(consumption):
    """Marginal utility CRRA"""
    
    mu = consumption**(-theta)
    
    return mu

def imutil(mutil):
    """Inverse marginal utility CRRA
    Consumption as a function of marginal utility"""
    
    cons = mutil**(-1/theta)
    
    return cons


def income(it, shock):
    """Income in period it given normal shock"""
    
    age = it + 20 # matlab strats counting at 1, Python at zero
    w = np.exp(inc0 + inc1*age - inc2*age**2 + shock)
    
    return w


def budget(it, savings, shocks, working):
    """Wealth, M_{t+1} in period t+1, where it == t
    
    Arguments
    ---------
        savings: np.array of savings with length ngridm
        shocks: np.array of shocks with length expn
    
    Returns
    -------
        w1: matrix with dimension (expn, ngridm) of all possible
    next period wealths
    """
    
    w1 = np.full((ngridm, expn), income(it + 1, shocks)*working).T + np.full((expn, ngridm), savings*(1+r))
    
    return w1

def mbudget():
    """Marginal budget:
    Derivative of budget with respect to savings"""
    
    mw1 = np.full((expn, ngridm), (1+r))
    
    return mw1

In [6]:
# Value function for worker
# interpolate and extrapolate are potentially substitutable by the interpolate function below

def value_function(working, it, x):
    """Value function calculation for the """
    
    x = x.flatten('F')
    
    res = np.full(x.shape, np.nan)
    
    # Mark constrained region
    mask = x < value[1, 0, working, it] # credit constraint between 1st (M_{t+1) = 0) and second point (A_{t+1} = 0)
    
    # Calculate t+1 value function in the constrained region
    res[mask] = util(x[mask], working) + df*value[0, 1, working, it]
    
    # Calculate t+1 value function in non-constrained region
    # interpolate
    res[~mask] = np.interp(x[~mask], value[:, 0, working, it], value[:, 1, working, it])
    # extrapolate
    slope = (value[-2, 1, working, it] - value[-1, 1, working, it])/(value[-2, 0, working, it] - value[-1, 0, working, it])
    intercept = value[-1, 1, working, it] - value[-1, 0, working, it]*slope
    res[res == np.max(value[:, 1, working, it])] = intercept + slope*x[res == np.max(value[:, 1, working, it])]

    return res

In [7]:
# Calculation of probability to choose work, if a worker today
def chpr(x):
    """Calculate the probability of choosing work in t+1
    for state worker given t+1 value functions"""
    
    mx = np.amax(x, axis = 0)
    mxx = x - mx
    res = np.exp(mxx[1, :]/lambda_)/np.sum(np.exp(mxx/lambda_), axis = 0)
    
    return res

In [8]:
# Expected value function calculation in state worker
def logsum(x):
    """Calculate expected value function"""
    
    mx = np.amax(x, axis = 0)
    mxx = x - mx
    res = mx + lambda_*np.log(np.sum(np.exp(mxx/lambda_), axis = 0))
    
    return res

m0 parametrisation - minimal shocks
--------------------------------------------

In [9]:
# Minimal shocks
sigma = 0
lambda_ = 2.2204e-16

In [10]:
# Initialize grids
quadp, quadw = quadrature(expn,0,1)
quadstnorm = scps.norm.ppf(quadp)
savingsgrid = np.linspace(0, mmax, ngridm)

In [11]:
# Initialize containers

# Container for endogenous gridpoints of (beginning-of-period) assets
# and corresponding consumption
policy = np.full((ngridm + 1, 2, 2, Tbar), np.nan)

# Value functions
value = np.full((ngridm + 1, 2, 2, Tbar), np.nan)

In [12]:
# Handling of last period and first elements
# policy
policy[1:, 0, 0, Tbar-1] = savingsgrid
policy[1:, 0, 1, Tbar-1] = savingsgrid
policy[1:, 1, :, Tbar-1] = policy[1:, 0, :, Tbar-1]
policy[0, :, :, :] = 0.00

In [13]:
# value
value[2:, 0, :, Tbar-1] = util(policy[2:, 0, :, Tbar-1], 0)
value[2:, 1, :, Tbar-1] = util(policy[2:, 0, :, Tbar-1], 1)
value[0:2, :, :, Tbar -1] = 0.00
value[0, 0, :, :] = 0.00

In [14]:
# Solve workers problem with EGM for period T-1, T-2 and T-3
# The EGM step already yields the same result as the matlab code for T-1 and T-2
# Difference in result for T-3 => DC step has to be performed after the EGM step
for period in [23, 22, 21]:
    
    for choice in[0, 1]:

        # M_{t+1}
        wk1 = budget(period, savingsgrid, quadstnorm*sigma, choice)
        wk1[wk1 < cfloor] = cfloor

        # Value function
        vl1 = np.full((2, ngridm * expn), np.nan)

        if period + 1 == Tbar - 1:
            vl1[0, :] = util(wk1, 0).flatten('F')
            vl1[1, :] = util(wk1, 1).flatten('F')
        else:
            vl1[1, :] = value_function(1, period + 1, wk1) # value function in t+1 if choice in t+1 is work
            vl1[0, :] = value_function(0, period + 1, wk1) # value function in t+1 if choice in t+1 is retiree

        # Probability of choosing work in t+1
        if choice == 0:
            # Probability of choosing work in t+1
            pr1 = np.full(2500, 0.00)
        else:
            pr1 = chpr(vl1)

        # Next period consumption based on interpolation and extrapolation
        # given grid points and associated consumption
        cons10 = np.interp(wk1, policy[:, 0, 0, period + 1], policy[:, 1, 0, period+1])
        # extrapolate linearly right of max grid point
        slope = (policy[-2, 1, 0, period + 1] - policy[-1, 1, 0, period + 1])/(policy[-2, 0, 0, period + 1] - policy[-1, 0, 0, period + 1])
        intercept = policy[-1, 1, 0, period + 1] - policy[-1, 0, 0, period + 1]*slope
        cons10[cons10 == np.max(policy[:, 1, 0, period+1])] = intercept + slope*wk1[cons10 == np.max(policy[:, 1, 0, period+1])]
        cons10_flat = cons10.flatten('F')

        cons11 = np.interp(wk1, policy[:, 0, 1, period + 1], policy[:, 1, 1, period+1])
        # extrapolate linearly right of max grid point
        slope = (policy[-2, 1, 1, period + 1] - policy[-1, 1, 1, period + 1])/(policy[-2, 0, 1, period + 1] - policy[-1, 0, 1, period + 1])
        intercept = policy[-1, 1, 1, period + 1] - policy[-1, 0, 1, period + 1]*slope
        cons11[cons11 == np.max(policy[:, 1, 1, period+1])] = intercept + slope*wk1[cons11 == np.max(policy[:, 1, 1, period+1])]
        cons11_flat = cons11.flatten('F')

        # Marginal utility of expected consumption next period
        mu1 = pr1*mutil(cons11_flat) + (1 - pr1)*mutil(cons10_flat)

        # Marginal budget
        # Note: Constant for this model formulation (1+r)
        mwk1 = mbudget()

        # RHS of Euler eq., p 337, integrate out error of y
        rhs = np.dot(quadw.T, np.multiply(mu1.reshape(wk1.shape, order = 'F'), mwk1))

        # Current period consumption from Euler equation
        cons0 = imutil(df*rhs)

        # Update containers related to consumption
        policy[1:, 1, choice, period] = cons0
        policy[1:, 0, choice, period] = savingsgrid + cons0


        if choice == 1:
            # Calculate continuation value
            ev = np.dot(quadw.T, logsum(vl1).reshape(wk1.shape, order = 'F'))
        else:
            ev = np.dot(quadw.T, vl1[0, :].reshape(wk1.shape, order = 'F'))

        # Update value function related containers
        value[1:, 1, choice, period] = util(cons0, choice) + df*ev
        value[1:, 0, choice, period] = savingsgrid + cons0
        value[0, 1, choice, period] = ev[0]

In [15]:
with open('m0_value.pkl', 'rb') as file : 
    m0_value = pickle.load(file)

with open('m0_policy.pkl', 'rb') as file : 
    m0_policy = pickle.load(file)

In [16]:
# Verify that EGM already yields correct solution for t-1 and t-2 in value
np.testing.assert_almost_equal(m0_value[0:501, 0, 0, 22], value[:, 1, 1, 22])

In [17]:
np.testing.assert_almost_equal(m0_value[0:501, 0, 1, 22], value[:, 1, 0, 22])

In [18]:
# Verify that EGM already yields correct solution for t-1 and t-2 in policy
np.testing.assert_almost_equal(m0_policy[0:501, 0, 0, 22], policy[:, 1, 1, 22])

In [19]:
np.testing.assert_almost_equal(m0_policy[0:501, 0, 1, 22], policy[:, 1, 0, 22])

In [20]:
# Difference in t-3
value[0:100, 1, 1, 21]

array([2.05931703, 2.45149424, 2.45635048, 2.461171  , 2.46588366,
       2.47055756, 2.47519777, 2.47973316, 2.4842344 , 2.48869924,
       2.49307278, 2.49741036, 2.50170953, 2.50593035, 2.51011269,
       2.5142557 , 2.51833174, 2.52236666, 2.52636232, 2.53030097,
       2.53419579, 2.53805225, 2.54186039, 2.54562197, 2.54934679,
       2.55303086, 2.55666561, 2.44271185, 2.45301246, 2.46308283,
       2.47297534, 2.4827064 , 2.49223383, 2.50159731, 2.51080362,
       2.51983117, 2.5287087 , 2.53743052, 2.5459949 , 2.55442686,
       2.56270043, 2.57083315, 2.57885725, 2.58671531, 2.59444954,
       2.60209492, 2.60956709, 2.61693268, 2.62422575, 2.63133893,
       2.63836114, 2.6453203 , 2.65210264, 2.65880848, 2.66545529,
       2.67193087, 2.67834133, 2.68469649, 2.6908866 , 2.69702015,
       2.70309779, 2.70902657, 2.71490019, 2.72071481, 2.7264027 ,
       2.73203201, 2.73760106, 2.74306257, 2.74846197, 2.75380139,
       2.75904988, 2.76423268, 2.76935705, 2.77440485, 2.77938

In [21]:
m0_value[18:118, 0, 0, 21]

array([2.05931703, 2.45149424, 2.45635048, 2.461171  , 2.46588366,
       2.47055756, 2.47519777, 2.47973316, 2.4842344 , 2.48869924,
       2.49307278, 2.49741036, 2.50170953, 2.50593035, 2.51011269,
       2.51034068, 2.51080362, 2.51983117, 2.5287087 , 2.53743052,
       2.5459949 , 2.55442686, 2.56270043, 2.57083315, 2.57885725,
       2.58671531, 2.59444954, 2.60209492, 2.60956709, 2.61693268,
       2.62422575, 2.63133893, 2.63836114, 2.6453203 , 2.65210264,
       2.65880848, 2.66545529, 2.67193087, 2.67834133, 2.68469649,
       2.6908866 , 2.69702015, 2.70309779, 2.70902657, 2.71490019,
       2.72071481, 2.7264027 , 2.73203201, 2.73760106, 2.74306257,
       2.74846197, 2.75380139, 2.75904988, 2.76423268, 2.76935705,
       2.77440485, 2.77938333, 2.78430602, 2.78916452, 2.79395007,
       2.79868337, 2.8033631 , 2.80796629, 2.81252148, 2.81702892,
       2.82146286, 2.82585035, 2.8301936 , 2.83446838, 2.83869778,
       2.84288528, 2.84700937, 2.85108957, 2.85512919, 2.85911

In [22]:
policy[:, 0, 1, period]

array([ 0.        ,  5.52589268,  5.66298055,  5.80006842,  5.93715629,
        6.07424416,  6.21133203,  6.3484199 ,  6.48550777,  6.62259564,
        6.75968351,  6.89677138,  7.03385925,  7.17094712,  7.308035  ,
        7.44512287,  7.58221074,  7.71929861,  7.85638648,  7.99347435,
        8.13056222,  8.26765009,  8.40473796,  8.54182583,  8.6789137 ,
        8.81600157,  8.95308944,  6.36285742,  6.49994529,  6.63703316,
        6.77412103,  6.9112089 ,  7.04829677,  7.18538464,  7.32247251,
        7.45956038,  7.59664825,  7.73373612,  7.87082399,  8.00791186,
        8.14499974,  8.28208761,  8.41917548,  8.55626335,  8.69335122,
        8.83043909,  8.96752696,  9.10461483,  9.2417027 ,  9.37879057,
        9.51587844,  9.65296631,  9.79005418,  9.92714205, 10.06422992,
       10.20131779, 10.33840566, 10.47549353, 10.61258141, 10.74966928,
       10.88675715, 11.02384502, 11.16093289, 11.29802076, 11.43510863,
       11.5721965 , 11.70928437, 11.84637224, 11.98346011, 12.12

Start secondary evelope
-----------------------------

To Do: Secondary envelope as a function and not line-by-line
To Do: Handling of discontinuity in the credit constrained region, retirement_model.m 137-148

In [23]:
obj = value[1:, :, 1, 21].T
obj.shape

(2, 500)

In [24]:
cur = obj

In [25]:
# Find discontinutiy
ii=cur[0][1:]>cur[0][:-1]

In [26]:
# Discontinuity at index 25 - 26
print(ii[25], ii[26])

False True


In [27]:
# Discontinuity in values
obj[0, 24:50]

array([8.81600157, 8.95308944, 6.36285742, 6.49994529, 6.63703316,
       6.77412103, 6.9112089 , 7.04829677, 7.18538464, 7.32247251,
       7.45956038, 7.59664825, 7.73373612, 7.87082399, 8.00791186,
       8.14499974, 8.28208761, 8.41917548, 8.55626335, 8.69335122,
       8.83043909, 8.96752696, 9.10461483, 9.2417027 , 9.37879057,
       9.51587844])

In [28]:
i = 1
sect = []

In [29]:
# Chop used in while loop below
# Does not work in original version - look at commented out and changes lines!
def chop(obj, j, repeat=None):
    
    #for k in range(len(obj)):
    for k in range(1):
        #if j > len(obj[k]):
            #j = len(obj[k])
        #result_1 = polyline(obj[k][0][:j], obj[k][1][:j])
        result_1 = polyline(obj[0][:j+1], obj[1][:j+1])
        result_2 = []
        
        if repeat != None:
            if repeat:
                #result_2 = polyline(obj[k][0][j:], obj[k][1][j:])
                result_2 = polyline(obj[0][j:], obj[1][j:])
            else:
                #result_2 = polyline(obj[k][0][j+1:], obj[k][0][j+1:])
                result_2 = polyline(obj[0][j+1:], obj[0][j+1:])
                
    return result_1, result_2

In [30]:
# Polyline used in while loop below
def polyline(x, y=None, label=None):
    """This function converts the """
    # create output container
    obj = np.empty((2,len(x)))
    
    # condition output on the number on function inputs 
    if x is not None:
        x = np.array(x).flatten()
        obj[0,:] = x
    if y is not None:
        y = np.array(y).flatten()
        obj[1,:] = y
        
    return obj

In [31]:
# Substitute for matlab while true loop
j = np.where([ii[counter] != ii[0] for counter in range(len(ii))])[0]

while len(j) > 0:
    j = [min(j)]

    for k in j:
        sect_container, cur = chop(cur, k, True)
        sect += [sect_container]
        ii = ii[k:]
        i +=1
        j = np.where([ii[counter] != ii[0] for counter in range(len(ii))])[0]

if i >1:
    sect += [cur]

In [32]:
# Note: One discontinutiy produces 3 sections
len(sect)

3

sect = sort_polyline(sect)

!!! Does not work with this object

In [33]:
# Explore wether one can simply use numpy for sorting insted of the translated matlab function
sect[1] = np.sort(sect[1])
sect[1]

array([[6.36285742, 8.95308944],
       [2.44271185, 2.55666561]])

Upper envelope

In [34]:
# aux_function used in upper_evelope below
def aux_function(x, obj1, obj2):
    x = [x]
    value, extr = np.subtract(interpolate(x,obj1), interpolate(x, obj2))
    return value

In [35]:
# interpolate used in upper_evelope below
def interpolate(xx, obj, one=False):    
    if not one:
        interpolation = InterpolatedUnivariateSpline(obj[0], obj[1], k=1)
        container = interpolation(xx)
        extrapolate = [True if (i>max(obj[0])) |(i<min(obj[0])) else False for i in xx]
    else:
        container = []
        extrapolate = []
        
        for poly  in obj:
            interpolation = InterpolatedUnivariateSpline(poly[0], poly[1], k=1)
            container += [interpolation(xx)]
            extrapolate += [np.array([True if (i>max(poly[0])) |(i<min(poly[0])) else False for i in xx])]
    return container, extrapolate

In [36]:
# Upper envelope changed compared to original
# Most importantly, assigning the proper output values to result_upper was completely missing
def upper_envelope(obj, fullinterval=False, intersection=False):
    
    # Assert if input of polyline objects is not an array (length==1)
    assert (len(obj) != 1), 'Upper envelope is meant for an array of polylines'
    length = []
    
    # copy original input
    aux_object = deepcopy(obj)
    
    # check length of polyline entries and drop polylines with x-length == 0
    for k1 in range(len(obj)):
        length += [(len(obj[k1][0]), len(obj[k1][1]))]
        aux_object[k1] = [i for i in aux_object[k1] if len(i) != 0]
    
    # Get all unique values of x
    xx = np.array([])
    for k1 in range(len(aux_object)):
           xx = np.append(xx, aux_object[k1][0].astype(list))
    xx =  np.array([i for i in np.unique(xx)])
    
    # set up containers
    interpolated = np.empty((len(obj), len(xx))) 
    extrapolated = np.empty((len(obj), len(xx)))
    
    # interpolate for each unique value of x
    for counter in range(len(obj)):
        inter, extra = interpolate(xx, obj[counter])
        interpolated[counter, :] = inter
        extrapolated[counter, :] = extra
    extrapolated = extrapolated.astype(bool)
    
    if not fullinterval:
        mask = np.sum(extrapolated, axis=0) > 0
        container = np.empty((interpolated.shape[0], int(mask.sum())))
        for i in range(interpolated.shape[0]):
            container[i,:] = np.extract(mask, interpolated[i,:])
        interpolated = container
        xx = xx[mask]
        n = sum(~mask)
    else:
        interpolated[extrapolated] = -np.inf
        n = len(xx)
        
    # create upper envelope
    maxinterpolated = repmat(interpolated.max(axis=0), m=interpolated.shape[0], n=1)
    top=interpolated==maxinterpolated 
    
    
    # Initialise container
    #result_upper = polyline(xx, maxinterpolated[1,:]) ## This does not seem to work as intended
    result_inter = np.empty((2,0))
    container1 = np.array([])
    container2 = np.array([])
    
    # Containers for collection of valid polyline points
    result_upper_cont_x = [xx[0]] # Added line
    result_upper_cont_y = [interpolated [0, 0]] #Added line

    Test = True
    if Test is True:
        k0 = np.where(top[:,0]==True)[0][0]
        for i in range(1, n):
            k1 = np.where(top[:,i]==True)[0][0]
            if k1 != k0:
                ln1 = k0
                ln2 = k1
                xx1 = xx[i - 1]
                xx2 = xx[i]
                y1, extr1 = interpolate([xx1, xx2], aux_object[ln1])
                y2, extr2 = interpolate([xx1, xx2], aux_object[ln2])
                if all(~np.stack([extr1, extr2])) & all(abs(y1-y2)>0):
                    xx3 = brenth(aux_function, xx1, xx2, args=(aux_object[ln1], aux_object[ln2]))
                    xx3f, _ = interpolate([xx3], aux_object[ln1])
                    # set up containers
                    interpolated2 = np.empty((len(obj), 1)) 
                    extrapolated2 = np.empty((len(obj), 1))

                    # interpolate for each unique value of x
                    for counter in range(len(obj)):
                        inter2, extra2 = interpolate([xx3], obj[counter])
                        interpolated2[counter] = inter2
                        extrapolated2[counter] = extra2

                    extrapolated2 = extrapolated2.astype(bool)
                    interpolated2[extrapolated2] = -np.inf
                    maxinterpolated2 = repmat(interpolated2.max(), m=len(obj), n=1)
                    ln3=np.where(interpolated2==maxinterpolated2)[0][0]
                    if (ln3==ln1) | (ln3==ln2):

                        #there are no other functions above!
                        #add the intersection point
                        result_upper_cont_x.append(xx3)
                        result_upper_cont_y.append(float(xx3f))
                        
                        if intersection:
                            container1 = np.append(container1, [xx3])
                            
                            container2 = np.append(container2, [xx3f])
                        if ln2==k1:

                            pass
                            
                        else:
                            ln1=ln2
                            xx1=xx3
                            ln2=k1
                            xx2=xx[i]
                    else:
                        ln2=ln3
                        xx2=xx3
            
            # This was missing before!!!! : replicates MatLab lines 342-346 which are extremely important
            # Add point to container if it is on the currently highest line            
            if any(abs(obj[k1][0] - xx[i]) < 2.2204e-16) == True:
                result_upper_cont_x.append(xx[i])
                result_upper_cont_y.append(maxinterpolated[0, i])
            
            k0=k1
        
        # Collect results
        result_inter = np.empty((2, len(container1)))
        result_inter[0], result_inter[1] = container1, container2
        result_upper = [np.array(result_upper_cont_x), np.array(result_upper_cont_y)] # Added line
        
    return result_upper, result_inter      

In [37]:
# Perform upper envelope calculation
result_container, newdots_container = upper_envelope(sect, True, True)

In [38]:
# Array of same length as MatLab code
# Same number of points removed
result_container[0].shape

(482,)

In [39]:
# Verify result for x values of value after upper envelope
# Same points removed as in MatLab code
np.testing.assert_array_almost_equal(m0_value[19:501, 1, 0, 21], result_container[0])

In [40]:
# Verify result for y values of value after upper envelope
# Same points removed as in MatLab code
np.testing.assert_array_almost_equal(m0_value[19:501, 0, 0, 21], result_container[1])

In [41]:
# Verify output of result_inter
# One new point added, one intersection, same as MatLab code
newdots_container

array([[7.31557897],
       [2.51034068]])

Finish up secondary envelope

diff(obj, result_container)

!!! Function not working with this input

In [42]:
# Find indexes of missing elements
missing_elements = np.setdiff1d(obj[0], result_container[0])
missing_elements

array([6.36285742, 6.49994529, 6.63703316, 6.77412103, 6.9112089 ,
       7.04829677, 7.18538464, 7.44512287, 7.58221074, 7.71929861,
       7.85638648, 7.99347435, 8.13056222, 8.26765009, 8.40473796,
       8.54182583, 8.6789137 , 8.81600157, 8.95308944])

In [43]:
indexremoved = []

for value in missing_elements:
    indexremoved.append(obj[0].tolist().index(value))

indexremoved = np.array(sort(indexremoved))

In [44]:
# Same indexes of removed points as in MatLab code
indexremoved

array([14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
       31, 32])

Back to retirement_model.m solve_dsegm line 149ff.

In [45]:
len(indexremoved) > 0

True

In [46]:
# All points below
# Note what MatLab function find is doing
# If one simply does "<" in Python, result would be wrong
# Current workaround might not be robust to all cases - find a better wat to pythonise MatLabs find function
j = arange(0, (np.where(policy[:, 0, 1, 21] > newdots_container[0][0]))[0][0])
j

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [47]:
# Points that were not deleted
j_new = np.setdiff1d(j, indexremoved)
j_new

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

In [48]:
j = max(j_new)
j

13

In [49]:
# Potentially a better way of setting j exists, such that indexes here are simplified
new_left = np.interp(newdots_container[0][0], policy[j+1:j+3, 0, 1, 21], policy[j+1:j+3, 1, 1, 21])
new_left

6.007459709059365

In [50]:
# Perform similar operation for the upper/right side
# All comments from above apply here
j = np.arange(np.where(policy[:, 0, 1, 21] < newdots_container[0][0])[0][-1], policy.shape[0]+1)
j

array([ 33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,
        46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,
        59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,
        72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
        85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
        98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110,
       111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123,
       124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136,
       137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149,
       150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162,
       163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175,
       176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188,
       189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201,
       202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 21

In [51]:
j_new = np.setdiff1d(j, indexremoved)
j_new

array([ 33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,
        46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,
        59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,
        72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
        85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
        98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110,
       111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123,
       124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136,
       137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149,
       150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162,
       163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175,
       176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188,
       189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201,
       202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 21

In [52]:
j = min(j_new)
j

33

In [53]:
new_right = np.interp(newdots_container[0][0], policy[j:j+2, 0, 1, period], policy[j:j+2, 1, 1, period])
new_right

4.014004377221639

In [62]:
# Remove inferior points from policy
# Means: Remove all points with indexes in indexremoved
policy_thinout_x = policy[:, 0, 1, period].tolist()
policy_thinout_y = policy[:, 1, 1, period].tolist()

In [64]:
del policy_thinout_x[indexremoved[0]+1 : indexremoved[-1]+2]
del policy_thinout_y[indexremoved[0]+1 : indexremoved[-1]+2]

In [65]:
# Add new point twice
policy_thinout_x.append(newdots_container[0][0] - 1e3*2.2204e-16)
policy_thinout_x.append(newdots_container[0][0])

In [66]:
# Add new point twice
policy_thinout_y.append(new_left)
policy_thinout_y.append(new_right)

In [67]:
len(policy_thinout_x)

484

In [69]:
new_policy = np.full((2, ngridm + 1), np.nan)
new_policy[0, - len(policy_thinout_x):] = np.array(policy_thinout_x)
new_policy[1, - len(policy_thinout_y):] = np.array(policy_thinout_y)

In [70]:
# Ensure that points are in the right position
new_policy = np.sort(new_policy)

In [75]:
# Verify correct solution for t-3 in policy
np.testing.assert_almost_equal(m0_policy[17:501, 1, 0, 21], new_policy[0, 0:484])

In [76]:
# Finish up
policy[:, :, 1, period] = new_policy.T